In [38]:
import numpy as np 
import pandas as pd 

                                                        Методы решения уравнений переноса
Найти аналитическое и численное решение смешанной задачи для уравнения переноса в квадрате $0 \le x, t \le 1$ и сравнить их значения в одиннадцати 
равноудаленных точках в момент времени $t = 1$ 



      Задание 3.
Дифференциальная задача:
\begin{equation*}
 \begin{cases}
    \frac{du}{dt} + (t + 6)\frac{du}{dx} = 0, 0 < t \le 1, 0 < x \le 1 \\
    u(x, 0) = x, u(0, t) = -6t - 0.5t^2     
 \end{cases}
\end{equation*}
Разностная схема:
$$D_h = {(x_l, t^n) : x_l = lh, hL = 1, l = 0,...,L; t^0 = 0, t^{n + 1} = \sum \tau_k, n = 0,..., N - 1 }$$

\begin{equation*}
 \begin{cases}
      u_l^{n+1} = u_l^n + \frac{\tau_n}{6h}(\frac{\tau_n}{2} + t^n + 6)(2u_{l-3}^n - 9u_{l-2}^n + 18u_{l-1}^n - 11u_l^n) + \\
                + \frac{\tau_n^2}{2h^2}(t^n + 6)(\tau_n + t^n + 6)(-u_{l-3}^n + 4u_{l-2}^n - 5u_{l-1}^n + 2u_l^n) - \\
                - \frac{\tau_n^3}{6h^3}(t^n + 6)^3(-u_{l-3}^n + 3u_{l-2}^n - 3u_{l-1}^n + u_l^n), l = 3,...,L; n = 0,...,N-1 \\
 \end{cases}
\end{equation*}


        Аналитическое решение 
Первые интеграллы:
$$dt = \frac{dx}{t + 6}$$
$$\frac{du} = 0$$
Следовательно их решения:
$$v_1 = t^2 / 2 + 6t - x$$
$$v_2 = u$$
Соответственно в $t = 0$:
$$x = -v_1$$
$$u = v_2$$
Подставив $x, u$ в первое начальное условие получаем:
$$u = x - t^2 / 2 - 6t$$
Соответственно в $x = 0$:
$$t = -v_1 / 4$$
$$u = -v_2$$
Подставив $t, u$ в первое начальное условие получаем:
$$u = x - t^2 / 2 - 6t$$
Тогда аналитическое решение выглядит, вот так:
$$u = x - t^2 / 2 - 6t$$

In [39]:

def analitical_solution(x, t):
    return x - 0.5 * t**2 - 6 * t

        Численное решение

In [40]:
def grid(L, N):
    '''
    returns np.arrays of x, t broken into L and N pieces
    '''
    return np.linspace(0, 1, L), np.linspace(0, 1, N)

#начальные условия
def phi(x):
    return x

def psi(t):
    return - 0.5 * t**2 - 6 * t

def u_x(t):
    return 1

def u_xx(t):
    return 0

def u_xxx(t):
    return 0

def numerical_solve(x_n, t_n, L, M, k):
    h = 1 / (L - 1)
    tau = h * k
    u = [x[:] for x in [[0] * L] * M]
    u[0] = phi(x_n)
    psi_m = psi(t_n)
    u_x_m = u_x(t_n)
    u_xx_m = u_xx(t_n)
    u_xxx_m = u_xxx(t_n)
    for m in range(1, M):
        u[m][0] = psi_m[m]
        u[m][1] = psi_m[m] + u_x_m * h + u_xx_m * h**2 / 2 + u_xxx_m * h**3 / 6
        u[m][2] = psi_m[m] + u_x_m * 2 * h + u_xx_m * h**2 * 2 + u_xxx_m * (h**3) * 4 / 3
    for m in range(M - 1):
        for l in range(3, L):
            u[m + 1][l] = u[m][l] + (2 * u[m][l - 3] - 9 * u[m][l - 2] + 18 * u[m][l - 1] - 11 * u[m][l]) * (tau * 0.5 + t_n[m] + 6) * tau / (6 * h) + (-u[m][l - 3] + 4 * u[m][l - 2] - 5 * u[m][l - 1] + 2 * u[m][l]) * (t_n[m] + 6) * (tau * 0.5 + t_n[m] + 6) * (tau**2) / (2 * h**2) - (-u[m][l - 3] + 3 * u[m][l - 2] - 3 * u[m][l - 1] + u[m][l]) * (t_n[m] + 6)**3 * (tau**3) / (6 * h**3) 
            
    return u

        Вывод решения

In [54]:
#находим аналитическое решение
x_rep = np.linspace(0, 1, 11)
u_analytical = analitical_solution(x_rep, 1)

#находим численное рашение
k = 0.4
L = 1281
M = int((L - 1) / k) + 1
x, t = grid(L, M)
u_n = numerical_solve(x, t, L, M, k)
u_numeric = np.array(u_n[-1][::int((len(u_n[0]) - 1) / 10)])

# запись данных в словарь и передча его в DataFrame для красоты
dat = {
    'x' : x_rep,
    'Analytical solve' : u_analytical,
    'Numeric solve' : u_numeric,
    'Error' : np.abs(u_analytical - u_numeric),
    'Max error' : np.max(np.abs(u_analytical - u_numeric))
}
df = pd.DataFrame(data=dat)
df.transpose()

,0,1,2,3,4,5,6,7,8,9,10
x,0.000000e+00,1.000000e-01,2.000000e-01,3.000000e-01,4.000000e-01,5.000000e-01,6.000000e-01,7.000000e-01,8.000000e-01,9.000000e-01,1.000000e+00
Analytical solve,-6.500000e+00,-6.400000e+00,-6.300000e+00,-6.200000e+00,-6.100000e+00,-6.000000e+00,-5.900000e+00,-5.800000e+00,-5.700000e+00,-5.600000e+00,-5.500000e+00
Numeric solve,-6.500000e+00,-6.400000e+00,-6.300000e+00,-6.200000e+00,-6.100000e+00,-6.000000e+00,-5.900000e+00,-5.800000e+00,-5.700000e+00,-5.600000e+00,-5.500000e+00
Error,0.000000e+00,1.065814e-13,1.856293e-13,2.051692e-13,1.989520e-13,4.592771e-12,6.391332e-12,7.228884e-12,1.328466e-10,4.045591e-10,9.362582e-10
Max error,9.362582e-10,9.362582e-10,9.362582e-10,9.362582e-10,9.362582e-10,9.362582e-10,9.362582e-10,9.362582e-10,9.362582e-10,9.362582e-10,9.362582e-10
